In [1]:
import bittensor as bt
import storage
import redis
import base64
import json
import sys
import os

bt.trace()

db = redis.Redis(host="localhost", port=6379, db=0)
db

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [2]:
# create a wallet instance that must be registered on the network
wallet = bt.wallet(name="default", hotkey="default")
print(wallet)

# instantiate the metagraph
metagraph = bt.metagraph(
    netuid=8, network="test", sync=True, lite=False
)
print(metagraph)

# Grab the axon you're serving
axon = metagraph.axons[13]
print(axon)

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite(wallet=wallet)
print(d)

wallet(default, default, ~/.bittensor/wallets/)
metagraph(netuid:8, n:65, block:619716, network:test)
AxonInfo( /ipv4/149.137.225.62:8091, 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH, 5GeYLB44QY9wcqJmFZvJW8D3EYPDaJGSgGfkbJVxUbkVcU7C, 610 )
dendrite(5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH)


In [3]:
# Setup CRS for initial round of validation
g, h = storage.shared.ecc.setup_CRS(curve="P-256")

# Make a bytes string to test the miner
data = b"this is a random bytestring, long enough to be chunked into segments and reconstructed at the end"

# Encrypt the data
encrypted_data, encryption_payload = storage.validator.encryption.encrypt_data(
    data,
    wallet,
)
encrypted_data

b'\x87\xc1\xbc\x06@u\x95\xb3o\xd4\x0c\x95\xfb[\r\xb5+\xd9,>\x0f\xd2\xd8l\xd5\xc1<\xdbz^\xc0v\x9dA\xb9\xf5\xc59\x04\x01\x7f\x87\x82Ul\xeb\xd3Y\xdd\x8d\xbb\x19\xcc`\x9b\x0cL\t"C\xbc7\xa3\xbb\x1bB\x88\x10\x8d\xd1^A\xa8\xe5\xffA\x12\x14\xbbG\xe1_*r\x1f\x97\x02vB\x16\xd2\t\xe9\xdc\xcc\xe4>'

In [4]:
data_hash = storage.shared.ecc.hash_data(encrypted_data)
print("data_hash:", data_hash)

encrypted_data

data_hash: 20192024516852024298847601818161253952637065606622713604040224283723139441518


b'\x87\xc1\xbc\x06@u\x95\xb3o\xd4\x0c\x95\xfb[\r\xb5+\xd9,>\x0f\xd2\xd8l\xd5\xc1<\xdbz^\xc0v\x9dA\xb9\xf5\xc59\x04\x01\x7f\x87\x82Ul\xeb\xd3Y\xdd\x8d\xbb\x19\xcc`\x9b\x0cL\t"C\xbc7\xa3\xbb\x1bB\x88\x10\x8d\xd1^A\xa8\xe5\xffA\x12\x14\xbbG\xe1_*r\x1f\x97\x02vB\x16\xd2\t\xe9\xdc\xcc\xe4>'

In [5]:
# Convert to base64 for compactness
b64_encrypted_data = base64.b64encode(encrypted_data).decode("utf-8")
b64_encrypted_data

'h8G8BkB1lbNv1AyV+1sNtSvZLD4P0ths1cE823pewHadQbn1xTkEAX+HglVs69NZ3Y27GcxgmwxMCSJDvDejuxtCiBCN0V5BqOX/QRIUu0fhXypyH5cCdkIW0gnp3MzkPg=='

In [6]:
# Create the synapse to store it
synapse = storage.protocol.Store(
    encrypted_data=b64_encrypted_data,
    data_hash=data_hash,
    curve="P-256",
    g=storage.shared.ecc.ecc_point_to_hex(g),
    h=storage.shared.ecc.ecc_point_to_hex(h),
    seed=os.urandom(32).hex(),
)
synapse

Store(required_hash_fields=['curve', 'g', 'h', 'seed', 'randomness', 'commitment', 'signature', 'commitment_hash'], encrypted_data='h8G8BkB1lbNv1AyV+1sNtSvZLD4P0ths1cE823pewHadQbn1xTkEAX+HglVs69NZ3Y27GcxgmwxMCSJDvDejuxtCiBCN0V5BqOX/QRIUu0fhXypyH5cCdkIW0gnp3MzkPg==', curve='P-256', g='31383532323531343834343331393830333231353130303232343930373337363933303235343937373932383337353435363232383938323634313638323237353835343630363139333139312c3938383837333837393637343631333135303437353033323733393739333831363432343538363839373638353232323535323535383337333437383835323438333337333730393532323235', h='35333635393534353036353831343036343432393539313536353831383437363831373737383531323033343035393135333835383939393532303837363030343432333637383133313735392c313130313739343933323630323432393633373230353635323632383938313638303331353835333832363538363439353432373634353436333237363731393430383630343832363730313038', seed='370700cabfb7e4dd615b53ed39bbfd569184c1d84f83c22cf9cf0f76132cc36e', randomness=

In [7]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    synapse,
    deserialize=False,
)
responses

2023-11-13 02:45:30.672 |      DEBUG       | dendrite | --> | 5021 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success
2023-11-13 02:45:30.710 |      DEBUG       | dendrite | <-- | 6218 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


[Store(required_hash_fields=['curve', 'g', 'h', 'seed', 'randomness', 'commitment', 'signature', 'commitment_hash'], encrypted_data='h8G8BkB1lbNv1AyV+1sNtSvZLD4P0ths1cE823pewHadQbn1xTkEAX+HglVs69NZ3Y27GcxgmwxMCSJDvDejuxtCiBCN0V5BqOX/QRIUu0fhXypyH5cCdkIW0gnp3MzkPg==', curve='P-256', g='31383532323531343834343331393830333231353130303232343930373337363933303235343937373932383337353435363232383938323634313638323237353835343630363139333139312c3938383837333837393637343631333135303437353033323733393739333831363432343538363839373638353232323535323535383337333437383835323438333337333730393532323235', h='35333635393534353036353831343036343432393539313536353831383437363831373737383531323033343035393135333835383939393532303837363030343432333637383133313735392c313130313739343933323630323432393633373230353635323632383938313638303331353835333832363538363439353432373634353436333237363731393430383630343832363730313038', seed='370700cabfb7e4dd615b53ed39bbfd569184c1d84f83c22cf9cf0f76132cc36e', randomness

In [8]:
resp = responses[0]
resp.dict()

{'name': 'Store',
 'timeout': 12.0,
 'total_size': 6218,
 'header_size': 0,
 'dendrite': {'status_code': 200,
  'status_message': 'Success',
  'process_time': 0.038819313049316406,
  'ip': '127.0.0.1',
  'port': 38208,
  'version': 610,
  'nonce': 10752285157787636,
  'uuid': 'b025ef54-81ce-11ee-9fa3-3ceceff2faf8',
  'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
  'signature': '0xb0a95edfdc06950d7e5f525411c86f366d5a51be2951a127ad77537f04c0fc58d0f25e2a6ba2a73cb65638175ab1b9cde65dde38fa5660af26a4feec0ce35989'},
 'axon': {'status_code': 200,
  'status_message': 'Success',
  'process_time': 0.009520769119262695,
  'ip': '149.137.225.62',
  'port': 8091,
  'version': 610,
  'nonce': 10752285176109371,
  'uuid': '88afb13a-81ce-11ee-acda-3ceceff2faf8',
  'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
  'signature': '0x044f0962bc5096efe7db27186578faa483cc6b184e1f1ab8d4c0e2f957fcab12da91d37d44d47293e567dd1fab37edbadebd50ad9609b7ff6b673aedee5d7b88'},
 'computed_bod

In [24]:
# Store necessary data on the validator side
response_storage = {
    "size": sys.getsizeof(encrypted_data),
    "prev_seed": synapse.seed,
    "commitment_hash": resp.commitment_hash,  # contains the seed
    # these will be private to the validator, not stored in decentralized GUN db
    # just here now for testing purposes
    "encryption_payload": encryption_payload
}
response_storage

{'size': 130,
 'prev_seed': '370700cabfb7e4dd615b53ed39bbfd569184c1d84f83c22cf9cf0f76132cc36e',
 'commitment_hash': '22401124309269929695136204336705305315153615166743524792851801992055863296944',
 'encryption_payload': '{"nonce": "1e490b5952f0ce3ad4ccebdbfab5b5ac868141b02ae5d578", "ciphertext": "7e440cd64f50aeeebbbd77adafd47ad65d0d48eff458095a0a599f3c2fa8f0311e3f3e36c07c8fff1ebb3b0b017f62bb4c8eee73d7e6ef323fb552bb3d266290cac88a6c6bd80cd04a390f45690cdef77e72e706c0234f3f751689bed5d528577e8c35dada40970471e2d67568988c2fe4635511446360038d544cd3808631be23dae2ba4420c51e9401a3743fb6ddddff79df8fd58b8264907bc75b60bfcf77f935a960e84324872a223c914d73185ca8b4af28635a2f"}'}

In [25]:
# Store in the database according to the data hash and the miner hotkey
key = f"{data_hash}.{resp.axon.hotkey}"
print("validator storage key:", key)
db.set(key, json.dumps(response_storage).encode())

validator storage key: 20192024516852024298847601818161253952637065606622713604040224283723139441518.5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH


True

In [27]:
storage.validator.verify.verify_store_with_seed(resp)

True

In [34]:
# Setup a challenge to the miner with a new CRS and on data we just stored
g, h = storage.shared.ecc.setup_CRS(curve="P-256")
challenge = storage.protocol.Challenge(
    challenge_hash=data_hash,
    g=storage.shared.ecc.ecc_point_to_hex(g),
    h=storage.shared.ecc.ecc_point_to_hex(h),
    curve="P-256",
    seed=os.urandom(32).hex(),
    challenge_index=1,
    chunk_size=storage.validator.utils.get_random_chunksize(sys.getsizeof(data)) // 4, # bytes
)
challenge

Challenge(required_hash_fields=['commitment_hash', 'commitment_proof', 'commitment', 'data_chunk', 'randomness', 'merkle_proof', 'merkle_root'], challenge_hash='20192024516852024298847601818161253952637065606622713604040224283723139441518', challenge_index=1, chunk_size=68, g='33303532383234323838343431333734343031393030343230343536303737373137303135373635333832393332343031303237383630363436353430303333363330353539323836313233352c3735383132373033323236313631353138373532323337353239303433393133393737353536373637363936323730353838373939353032333333333535363639313234343237303134313731', h='3131303431373239353633373131373731353935383530313638333233373930373638333735303732383938353731393836363537333037353937313335393036303333333639303033343030302c3939323537383035373232353239353537363133363334313532373030363831343534323131313431333235303731353639323539393835393632313136323936303132313731333833333437', curve='P-256', seed='b1ae53e3d3e1c5dd85a3911fc2adaacb72d630c40efc3739ae5b8790c4ba426a', com

In [35]:
# Send the challenge to the miner
responses = d.query(
    [metagraph.axons[13]],
    challenge,
    deserialize=False,
)
responses[0]

2023-11-13 02:51:37.386 |      DEBUG       | dendrite | --> | 5930 B | Challenge | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success
2023-11-13 02:51:37.464 |      DEBUG       | dendrite | <-- | 7763 B | Challenge | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


Challenge(required_hash_fields=['commitment_hash', 'commitment_proof', 'commitment', 'data_chunk', 'randomness', 'merkle_proof', 'merkle_root'], challenge_hash='20192024516852024298847601818161253952637065606622713604040224283723139441518', challenge_index=1, chunk_size=68, g='33303532383234323838343431333734343031393030343230343536303737373137303135373635333832393332343031303237383630363436353430303333363330353539323836313233352c3735383132373033323236313631353138373532323337353239303433393133393737353536373637363936323730353838373939353032333333333535363639313234343237303134313731', h='3131303431373239353633373131373731353935383530313638333233373930373638333735303732383938353731393836363537333037353937313335393036303333333639303033343030302c3939323537383035373232353239353537363133363334313532373030363831343534323131313431333235303731353639323539393835393632313136323936303132313731333833333437', curve='P-256', seed='b1ae53e3d3e1c5dd85a3911fc2adaacb72d630c40efc3739ae5b8790c4ba426a', com

In [36]:
# Verify the challenge
storage.validator.verify.verify_challenge_with_seed(responses[0])

True

In [39]:
# Now retrieve the data we stored and successfully challenged
syn = storage.protocol.Retrieve(
    data_hash=data_hash,
    seed=os.urandom(32).hex(),
)
syn

Retrieve(required_hash_fields=['data', 'data_hash', 'seed', 'commtiment_proof', 'commitment_hash'], data_hash='20192024516852024298847601818161253952637065606622713604040224283723139441518', seed='cc505af5cf93d94bd26eeb508b5e486fefba4f4552d0cb77e6620e038b823a49', data=None, commitment_hash=None, commitment_proof=None)

In [40]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    syn,
    deserialize=False,
)
responses 

2023-11-13 02:53:58.457 |      DEBUG       | dendrite | --> | 4101 B | Retrieve | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success
2023-11-13 02:53:58.502 |      DEBUG       | dendrite | <-- | 5025 B | Retrieve | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


[Retrieve(required_hash_fields=['data', 'data_hash', 'seed', 'commtiment_proof', 'commitment_hash'], data_hash='20192024516852024298847601818161253952637065606622713604040224283723139441518', seed='cc505af5cf93d94bd26eeb508b5e486fefba4f4552d0cb77e6620e038b823a49', data='h8G8BkB1lbNv1AyV+1sNtSvZLD4P0ths1cE823pewHadQbn1xTkEAX+HglVs69NZ3Y27GcxgmwxMCSJDvDejuxtCiBCN0V5BqOX/QRIUu0fhXypyH5cCdkIW0gnp3MzkPg==', commitment_hash='13456453580021765705508015074074487849084226271130232772016458377822729844935', commitment_proof='13623431165461458479630801995263404700046767654466537693643932201000109609059')]

In [41]:
# extract the base64 encoded string of the data
resp = responses[0]
resp.data

'h8G8BkB1lbNv1AyV+1sNtSvZLD4P0ths1cE823pewHadQbn1xTkEAX+HglVs69NZ3Y27GcxgmwxMCSJDvDejuxtCiBCN0V5BqOX/QRIUu0fhXypyH5cCdkIW0gnp3MzkPg=='

In [42]:
# decode from base64 to bytes
decoded_data = base64.b64decode(resp.data)
decoded_data

b'\x87\xc1\xbc\x06@u\x95\xb3o\xd4\x0c\x95\xfb[\r\xb5+\xd9,>\x0f\xd2\xd8l\xd5\xc1<\xdbz^\xc0v\x9dA\xb9\xf5\xc59\x04\x01\x7f\x87\x82Ul\xeb\xd3Y\xdd\x8d\xbb\x19\xcc`\x9b\x0cL\t"C\xbc7\xa3\xbb\x1bB\x88\x10\x8d\xd1^A\xa8\xe5\xffA\x12\x14\xbbG\xe1_*r\x1f\x97\x02vB\x16\xd2\t\xe9\xdc\xcc\xe4>'

In [44]:
# decrypt back to the original file content bytes
storage.validator.encryption.decrypt_data(
    decoded_data,
    response_storage['encryption_payload'],
    wallet=wallet,
)

b'this is a random bytestring, long enough to be chunked into segments and reconstructed at the end'